In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')

In [22]:
wine_input = wine[['alcohol','sugar','pH']].to_numpy()
# 여러개니까 []안에 []리스트업
wine_target = wine['class'].to_numpy()

### 훈련세트와 테스트세트 분리

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    wine_input, wine_target, test_size = 0.2, random_state=42)

### 훈련세트에서 검증세트를 분리

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state=42)
# 최종적으로 6 : 2 : 2 로 분리한 것

In [6]:
sub_input.shape, val_input.shape
# 데이터 크기 확인

((4157, 3), (1040, 3))

### 모델 구축(DecisionTreeClassifier)

In [7]:
from sklearn.tree import DecisionTreeClassifier
#                                       분류
dt = DecisionTreeClassifier(random_state=42)
# 아무 옵션도 안줘서 최대 트리로 형성될꺼임
dt.fit(sub_input, sub_target)
# 검증세트를 학습시킬거임

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# sub >> val
# 과대적합 되어있다

0.9971133028626413
0.864423076923077


### 교차 검증(k-fold cross validation)

In [8]:
from sklearn.model_selection import cross_validate
# 보통 5나 10을 많이 써서 5겹 교차검증/10겹 교차검증이라고 많이 씀

# 평가할 모델 객체를 매개변수로 전달한다.(검정세트를 떼어내지 않고 훈련세트 전체를 전달해야 한다)
scores = cross_validate(dt, train_input, train_target)
# 매개변수 3개 들어감 주의
# 어떤 옵션(k값)을 안줬는데 디폴트는 5임
# 반복해서 검증하면서 평균값이 나오면 그걸로 평가점수를 반환해줌
# scores에 받기
scores
# 세개의 키값과 함께 딕셔너리를 반환해줌
# fit_time  훈련시간
# score_time 검증시간
# test_score 평균값
# 다섯겹으로 쪼갰기때문에 다섯개의 값이 나옴

{'fit_time': array([0.00897527, 0.00724292, 0.00645304, 0.00584793, 0.00590754]),
 'score_time': array([0.        , 0.00073814, 0.00113368, 0.00019026, 0.        ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [9]:
np.mean(scores['test_score'])
# 평균내면 최종_찐최종_점수

0.855300214703487

In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
# cv=StratifiedKFold() 얘를 넣어주면 알아서 섞어줌(shuffle기능)
np.mean(scores['test_score'])

0.855300214703487

In [11]:
# 만약 k-fold가 10이라면 (10겹 교차검증)
Splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# splitter를 쓸때는 이렇게 써줘야 함
# 한번섞어서 나누는 용
scores = cross_validate(dt, train_input, train_target, cv=Splitter)
np.mean(scores['test_score'])

0.8574181117533719

 특성공학을 이용해서 특성을 늘릴 필요성이 있다

### 와인의 그리드서치

In [16]:
wine.keys()

Index(['alcohol', 'sugar', 'pH', 'class'], dtype='object')

In [18]:
type(wine['class'])

pandas.core.series.Series

In [29]:
type(wine)

pandas.core.frame.DataFrame

In [26]:
wine.tail()

,alcohol,sugar,pH,class
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0
6496,11.8,0.8,3.26,1.0


In [34]:
import graphviz

from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier(random_state = 2022)

params = {
    'max_depth':[2, 3, 4, 5, None],
    'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_split':[2, 3, 4, 5, 6, 7, 8, 9, 10]
}
# None : 깊이 제한을 안준것

gs = GridSearchCV(model, param_grid = params, cv = 3, refit = True, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2022),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, None],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [32]:
model = DecisionTreeClassifier(min_samples_split=4, random_state=2022)
model.fit(train_input, train_target)

DecisionTreeClassifier(min_samples_split=4, random_state=2022)

In [35]:
model = gs.best_estimator_
# 우리가 가진 값중 최적의 값을 가지고 알아서 모델을 만들라는 뜻
print(model.score(train_input, train_target))
print(model.score(test_input, test_target))
## 과대적합의심!(왜? train데이터셋의 값이 test데이터셋의 값보다 많아서)

0.9045603232634212
0.8507692307692307


In [36]:
## 그리드 설치를 통해 찾은 최적의 매개 변수
print(gs.best_params_)
# 최적의 파라미터를 통해 찾은 최적의 점수가 무엇인가?
print(gs.best_score_) 

{'max_depth': None, 'min_samples_leaf': 8, 'min_samples_split': 2}
0.8649229488527506


In [37]:
#그리드 서치를 통해서 찾은 최상의 교차검증 점수
np.max(gs.cv_results_['mean_test_score'])

## grid-search = gs
## cross-validaion = cv

0.8649229488527506

In [38]:
model = DecisionTreeClassifier(max_depth = 3, min_samples_split=2,random_state=121)
model.fit(train_input, train_target)

print(model.score(train_input, train_target)) 

print(model.score(test_input, test_target))

# 값이 맞춰졌다는거는 나쁘지 않은것..

0.8454877814123533
0.8415384615384616


In [ ]:
## 과소적합일 때는 데이터가 적을 때 발생할 수 있는데,
## 이 때 해결방법은 데이터를 늘리거나 특성공학을 쓸 수 있다
##하지만, 이 데이터셋에서는 여기서 그칠 수밖에 없다.